In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
from pandas_plink import read_plink
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
from sklearn import metrics
from math import sqrt
import lime
import lime.lime_tabular
from lime import submodular_pick
import random
from sklearn.metrics import roc_curve,roc_auc_score

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
''' Creating arrays with optimal data structure and filling missing values with 2--> Homozygous major '''

print("Convertion")
bed=bed.astype('uint8')
print("Compute")
X=bed.compute()
print("Filling Null Data")
X[np.isnan(X)]=2
#validation
print("Convertion")
bed2=bed2.astype('uint8')
print("Compute")
X_val=bed2.compute()
print("Filling Null Data")
X_val[np.isnan(X_val)]=2
print("Convertion")
#test
bed3=bed3.astype('uint8')
print("Compute")
X_test=bed3.compute()
print("Filling Null Data")
X_test[np.isnan(X_test)]=2

''' Preparing data.shape=(individuals,SNP) '''

Y=fam["trait"].astype("int")
Y=Y-1
Xdf=pd.DataFrame(X.T)
Xdf["Y"]=Y

Y_val=fam2["trait"].astype("int")
Y_val=Y_val-1
Xdf_val=pd.DataFrame(X_val.T)
Xdf_val["Y"]=Y_val

Y_test=fam3["trait"].astype("int")
Y_test=Y_test-1
Xdf_test=pd.DataFrame(X_test.T)
Xdf_test["Y"]=Y_test

''' Getting np arrays '''

x_train=Xdf.drop(['Y'],axis=1).values
y_train=Xdf[['Y']].values
del Xdf

x_val=Xdf_val.drop(['Y'],axis=1).values
y_val=Xdf_val[['Y']].values
del Xdf_val

x_test=Xdf_test.drop(['Y'],axis=1).values
y_test=Xdf_test[['Y']].values
del Xdf_test

In [ ]:
'''Creating graph'''

In [ ]:
tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=(None,5273),name="X")
X_random=tf.placeholder(tf.float32,shape=(None,1),name="X")
Y=tf.placeholder(tf.float32,shape=(None,1),name="Y")

In [ ]:

with tf.name_scope("dnn"):
    training = tf.placeholder_with_default(False, shape=(), name='training')
    initializer = tf.contrib.layers.xavier_initializer()
    hidden00_drop= tf.layers.dropout(X, 0.5, training=training)
    hidden01=tf.layers.dense(hidden00_drop, 1000, name="hidden01",activation=None, kernel_initializer=initializer)
    hidden01_norm=tf.layers.batch_normalization(hidden01, training=training, momentum=0.9)
    act_hidden01=tf.nn.leaky_relu(hidden01_norm)
    hidden01_drop = tf.layers.dropout(act_hidden01, 0.5, training=training)
    hidden0=tf.layers.dense(hidden01_drop, 250, name="hidden0",activation=None, kernel_initializer=initializer)
    hidden0_norm=tf.layers.batch_normalization(hidden0, training=training, momentum=0.9)
    act_hidden0=tf.nn.leaky_relu(hidden0_norm)
    hidden0_drop = tf.layers.dropout(act_hidden0, 0.5, training=training)
    hidden1=tf.layers.dense(hidden0_drop, 50, name="hidden1",activation=None, kernel_initializer=initializer)
    hidden1_norm=tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
    act_hidden1=tf.nn.leaky_relu(hidden1_norm)
    hidden1_drop = tf.layers.dropout(act_hidden1, 0.5, training=training)
    output=tf.layers.dense(  hidden1_drop, 1, name="output_final",activation=None)


In [ ]:
with tf.name_scope("loss"):
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=output)
    weights = tf.trainable_variables()
    loss=tf.reduce_mean(cross_entropy)
    error=loss

In [ ]:
with tf.name_scope("train"):
    optimizer =tf.train.AdamOptimizer(learning_rate=0.0001,beta1=0.9,beta2=0.999,epsilon=1e-08,use_locking=False,name='Adam')
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = optimizer.minimize(error)

In [ ]:
with tf.name_scope("eval"):
    predicted = tf.nn.sigmoid(output)
    correct_pred = tf.equal(tf.round(predicted), Y)
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    _,auc = tf.metrics.auc(labels=Y,predictions=predicted)

In [ ]:
sess.close()

In [ ]:
saver=tf.train.Saver()


In [ ]:
'''Restaure the trained model. No need to train!'''

In [ ]:
sess = tf.InteractiveSession(config=tf.ConfigProto(device_count={ "CPU": 44}))


In [ ]:
saver.restore(sess, "/work/breastcancer/clean_test/logs/preprocessed001.ckpt")

In [ ]:
'''Getting individuals where precision > 90%'''

In [ ]:
pred=sess.run(predicted,feed_dict={X:x_test,Y:y_test})

In [ ]:
important=np.argwhere(pred==pred[pred>0.67])[:,0]

In [ ]:
'''Train explainer'''

In [ ]:
'''Pre-training (scaling and preparing LIME representation space...)'''
explainer = lime.lime_tabular.LimeTabularExplainer(x_train ,class_names=['not tumour', 'Tumor'],categorical_features = range(5273), feature_names = list(bim["snp"].values), kernel_width=40, verbose=True)

In [ ]:
def predict_fn(x):
    if len(x.shape)>1:
         return np.array([1-sess.run(tf.nn.sigmoid(output),feed_dict={X:x})[:,0],sess.run(tf.nn.sigmoid(output),feed_dict={X:x})[:,0]]).T
    else:
        return np.array([[1-sess.run(tf.nn.sigmoid(output),feed_dict={X:np.expand_dims(x,axis=1).T})[0,0],sess.run(tf.nn.sigmoid(output),feed_dict={X:np.expand_dims(x,axis=1).T})[0,0]]])

In [ ]:
'''submodular explanation of testing set'''
pick=submodular_pick.SubmodularPick(explainer, x_test[important], predict_fn, sample_size=553, num_features=41, num_exps_desired=1)

In [ ]:
'''Getting average contribution values over explanations'''

In [ ]:

contribution_values_per_explanation=pd.DataFrame([dict(this.as_list()) for this in pick.explanations])


In [ ]:
contribution_values_per_explanation.to_csv(paht_logs+"lime_contributions.csv")

In [ ]:
average_conributions=contribution_values_per_explanation.fillna(0).mean()

In [ ]:
fig,ax=plt.subplots(figsize=(20,5))

ax.plot(average_conributions)